### In this file i am implementing simple RAG architicture. ###

#### Fetch the doc ####
#### Split the docs, embed and store in vetor store ####
#### fectch the relivent docs and add them as context to the prompt ####


In [1]:
from src import get_azure_openai_model, get_azure_openai_mini_model

model = get_azure_openai_model()
model2 = get_azure_openai_mini_model()

### Embedding model

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

### Vector store

In [3]:
from langchain_chroma import Chroma

vector_store = Chroma(embedding_function=embeddings)

In [4]:
# Load PDFs
from langchain_community.document_loaders import PyPDFLoader
import os

folder_path = "papers"
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]

pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(os.path.join(folder_path, pdf_file))
    async for page in loader.alazy_load():
        pages.append(page)

CancelledError: 

In [13]:
pages[100]

Document(metadata={'source': 'papers\\Alexander Zai, Brandon Brown - Deep Reinforcement Learning in Action-Manning Publications (2020).pdf', 'page': 72}, page_content='51Predicting future rewards: Value and policy functions\nIn the mathematical notation, s is a state and Pr(A | s) is a probability distribution over\nthe set of actions A, given state s. The probability of each action in the distribution is\nthe probability that the action will produce the greatest reward.\n2.7.2 Optimal policy \nThe policy is the part of our reinforcement learning algorithm that chooses actions\ngiven its current state. We can then formulate the optimal policy —it’s the strategy that\nmaximizes rewards.\nRemember, a particular policy is a map or function, so we have some sort of set of pos-\nsible policies; the optimal policy is just an argmax (which selects the maximum) over\nthis set of possible policies as a function of their expected rewards.\n Again, the whole goal of a reinforcement learning algor

In [ ]:
# Add docs to vector store
# Index chunks
_ = vector_store.add_documents(documents=pages)

In [ ]:
# Vector search similarity test

docs = vector_store.similarity_search("What is the replay memory?", k=1)
for doc in docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

Page 97: 76 CHAPTER 3 Predicting the best states and actions: Deep Q-networks
3.3.2 Experience replay
Catastrophic forgetting is probably not something we have to worry about with the first
variant of our game because the targets are always stationary, and indeed the model
successfully learned how to play it



In [ ]:
# Retrival
retriever = vector_store.as_retriever()


In [ ]:
# This prompt provides instructions to the model. 
# The prompt includes the query and the source, which are specified further down in the code.
GROUNDED_PROMPT="""
You are a friendly assistant that helps students.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# The query is sent to the search engine, but it's also passed in the prompt
query="what is deeplearning?"

# Retrieve the selected fields from the search index related to the question
retrieved_docs = vector_store.similarity_search(query, k=2)
for doc in retrieved_docs:
    print(f'Page {doc.metadata["page"]}: {doc.page_content[:300]}\n')

Page 14: xiii
preface
Deep reinforcement learning was launched into the spotlight in 2015, when Deep-
Mind produced an algorithm capable of playing a suite of Atari 2600 games at super-
human performance. Artificial intelligence seemed to be finally making some real
progress, and we wanted to be a part of it

Page 14: xiii
preface
Deep reinforcement learning was launched into the spotlight in 2015, when Deep-
Mind produced an algorithm capable of playing a suite of Atari 2600 games at super-
human performance. Artificial intelligence seemed to be finally making some real
progress, and we wanted to be a part of it



In [ ]:


response = model2.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=retrieved_docs)
        }
    ],
    model="gpt-4o-mini-test"
)

print(response.choices[0].message.content)

- The term "deep learning" refers to a significant advancement in artificial intelligence, particularly evident since the deep learning revolution around 2012.
- Deep reinforcement learning gained attention in 2015 when DeepMind created an algorithm that outperformed humans in playing Atari 2600 games. 
- Deep learning combines machine learning techniques with neural network architectures, emphasizing its effectiveness in various applications.
- Individuals with software engineering backgrounds can understand the fundamentals of deep reinforcement learning despite the advanced mathematics involved.
